In [1]:
## This file implements neural networks and logistic regression on p003ppresabsSTCC_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 87.88% for combination data, and 84.72% for over-sampling data.
## To improve accuracy, we use lasso to select important features and thus have new training and test datasets.
## For the new neural networks on these new datasets, the accuracy is 83.46% for combination data and 83.33% for over-sampling data.
## For logistic regression, the accuracies are 100% for combination data and 97.92% for over-sampling data.
## Since the accuracy scores are pretty high in logistic regression, we further construct random forest models, which 
## are relatively less likely to bring overfitting compared to decision tree.
## For random forest, the accuracy is 99.24% for combination and 100% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 99.34% for combination data, and 98.51% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p003ppresabsSTCC_quant.csv')
df.shape

(255, 868)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.169500
1      0.316500
2      0.460000
3      0.169000
4      0.171333
5      0.236333
6      0.166500
7      0.173333
8      0.315833
9      0.162167
10     0.186167
11     0.382833
12     0.170667
13     0.173333
14     0.162833
15     0.164333
16     0.382667
17     0.397000
18     0.372667
19     0.166333
20     0.242333
21     0.305833
22     0.162333
23     0.165500
24     0.616833
25     0.170167
26     0.491500
27     0.167833
28     0.166500
29     0.374833
         ...   
225    0.267200
226    0.293800
227    0.383400
228    0.513500
229    0.507700
230    0.311500
231    0.236000
232    0.228833
233    0.219333
234    0.167333
235    0.265167
236    0.160500
237    0.159333
238    0.165500
239    0.327833
240    0.342000
241    0.308500
242    0.170333
243    0.165333
244    0.340833
245    0.162000
246    0.170500
247    0.164500
248    0.164000
249    0.180000
250    0.348000
251    0.232500
252    0.166167
253    0.276667
254    0.225333
Name: pheno, Length: 255

In [6]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [7]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [8]:
df['pheno']
df['pheno'].value_counts()

0    240
1     15
Name: pheno, dtype: int64

In [9]:
df.shape

(255, 868)

In [10]:
df_clean = df.drop(columns=['id'])

In [11]:
df_clean.shape

(255, 867)

In [12]:
df_clean.head()

,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,TTTTTCTTATTTTA,...,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9508,pheno,ST,CC
0,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,8,8
2,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5
3,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5
4,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,5,5


In [13]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 866) (255,)


In [14]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 209), (1, 228)]


In [15]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 240), (1, 240)]


In [16]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [17]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [18]:
############# Fully-Connected Neural Network ################

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [20]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [21]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 305 samples, validate on 132 samples
Epoch 1/1000
305/305 [==============================] - 1s 2ms/step - loss: 4.2886 - accuracy: 0.5607 - val_loss: 0.6215 - val_accuracy: 0.7727
Epoch 2/1000
305/305 [==============================] - 0s 155us/step - loss: 0.6221 - accuracy: 0.6885 - val_loss: 0.5893 - val_accuracy: 0.7652
Epoch 3/1000
305/305 [==============================] - 0s 144us/step - loss: 0.5997 - accuracy: 0.7443 - val_loss: 0.5555 - val_accuracy: 0.7879
Epoch 4/1000
305/305 [==============================] - 0s 133us/step - loss: 0.5824 - accuracy: 0.7377 - val_loss: 0.5594 - val_accuracy: 0.7045
Epoch 5/1000
305/305 [==============================] - 0s 109us/step - loss: 0.5648 - accuracy: 0.7574 - val_loss: 0.5307 - val_accuracy: 0.7803
Epoch 6/1000
305/305 [==============================] - 0s 118us/step - loss: 0.5723 - accuracy: 0.7508 - val_loss: 0.5231 - val_accuracy: 0.7727
Epoch 7/1000
305/305 [==============================] - 0s 114us/step - loss: 0.

305/305 [==============================] - 0s 164us/step - loss: 0.4163 - accuracy: 0.8525 - val_loss: 0.4214 - val_accuracy: 0.8106
Epoch 57/1000
305/305 [==============================] - 0s 150us/step - loss: 0.4152 - accuracy: 0.8557 - val_loss: 0.4038 - val_accuracy: 0.8409
Epoch 58/1000
305/305 [==============================] - 0s 131us/step - loss: 0.4081 - accuracy: 0.8590 - val_loss: 0.4118 - val_accuracy: 0.8030
Epoch 59/1000
305/305 [==============================] - 0s 171us/step - loss: 0.4068 - accuracy: 0.8525 - val_loss: 0.3925 - val_accuracy: 0.8561
Epoch 60/1000
305/305 [==============================] - 0s 159us/step - loss: 0.4037 - accuracy: 0.8590 - val_loss: 0.3864 - val_accuracy: 0.8712
Epoch 61/1000
305/305 [==============================] - 0s 101us/step - loss: 0.4092 - accuracy: 0.8492 - val_loss: 0.3871 - val_accuracy: 0.8636
Epoch 62/1000
305/305 [==============================] - 0s 122us/step - loss: 0.3968 - accuracy: 0.8623 - val_loss: 0.3850 - val_ac

Epoch 112/1000
305/305 [==============================] - 0s 138us/step - loss: 0.2884 - accuracy: 0.8656 - val_loss: 0.2872 - val_accuracy: 0.8864
Epoch 113/1000
305/305 [==============================] - 0s 117us/step - loss: 0.2769 - accuracy: 0.8918 - val_loss: 0.2837 - val_accuracy: 0.8636
Epoch 114/1000
305/305 [==============================] - 0s 131us/step - loss: 0.2822 - accuracy: 0.8820 - val_loss: 0.2869 - val_accuracy: 0.8788
Epoch 115/1000
305/305 [==============================] - 0s 113us/step - loss: 0.2799 - accuracy: 0.8754 - val_loss: 0.2752 - val_accuracy: 0.9015
Epoch 116/1000
305/305 [==============================] - 0s 118us/step - loss: 0.2753 - accuracy: 0.8885 - val_loss: 0.2870 - val_accuracy: 0.8636
Epoch 117/1000
305/305 [==============================] - 0s 126us/step - loss: 0.2773 - accuracy: 0.8820 - val_loss: 0.3030 - val_accuracy: 0.8788
Epoch 118/1000
305/305 [==============================] - 0s 143us/step - loss: 0.2886 - accuracy: 0.8885 - val_

305/305 [==============================] - 0s 90us/step - loss: 0.2434 - accuracy: 0.8820 - val_loss: 0.2293 - val_accuracy: 0.8864
Epoch 168/1000
305/305 [==============================] - 0s 91us/step - loss: 0.2362 - accuracy: 0.8754 - val_loss: 0.2458 - val_accuracy: 0.8712
Epoch 169/1000
305/305 [==============================] - 0s 95us/step - loss: 0.2326 - accuracy: 0.8918 - val_loss: 0.2466 - val_accuracy: 0.9167
Epoch 170/1000
305/305 [==============================] - 0s 109us/step - loss: 0.2367 - accuracy: 0.8951 - val_loss: 0.2297 - val_accuracy: 0.8788
Epoch 171/1000
305/305 [==============================] - 0s 86us/step - loss: 0.2191 - accuracy: 0.8984 - val_loss: 0.2850 - val_accuracy: 0.8258
Epoch 172/1000
305/305 [==============================] - 0s 74us/step - loss: 0.2608 - accuracy: 0.8689 - val_loss: 0.2488 - val_accuracy: 0.8712
Epoch 173/1000
305/305 [==============================] - 0s 77us/step - loss: 0.2415 - accuracy: 0.8852 - val_loss: 0.2467 - val_ac

305/305 [==============================] - 0s 120us/step - loss: 0.2327 - accuracy: 0.8689 - val_loss: 0.2214 - val_accuracy: 0.9015
Epoch 223/1000
305/305 [==============================] - 0s 109us/step - loss: 0.2205 - accuracy: 0.8885 - val_loss: 0.2390 - val_accuracy: 0.8182
Epoch 224/1000
305/305 [==============================] - 0s 91us/step - loss: 0.2171 - accuracy: 0.8984 - val_loss: 0.2311 - val_accuracy: 0.8788
Epoch 225/1000
305/305 [==============================] - 0s 101us/step - loss: 0.2132 - accuracy: 0.8852 - val_loss: 0.2236 - val_accuracy: 0.8788
Epoch 226/1000
305/305 [==============================] - 0s 87us/step - loss: 0.2215 - accuracy: 0.8820 - val_loss: 0.2237 - val_accuracy: 0.8788
Epoch 227/1000
305/305 [==============================] - 0s 97us/step - loss: 0.2190 - accuracy: 0.8885 - val_loss: 0.2151 - val_accuracy: 0.9167
Epoch 228/1000
305/305 [==============================] - 0s 77us/step - loss: 0.2149 - accuracy: 0.8918 - val_loss: 0.2417 - val_

...

305/305 [==============================] - 0s 123us/step - loss: 0.2220 - accuracy: 0.8754 - val_loss: 0.2071 - val_accuracy: 0.8788
Epoch 773/1000
305/305 [==============================] - 0s 123us/step - loss: 0.1979 - accuracy: 0.8852 - val_loss: 0.2058 - val_accuracy: 0.8788
Epoch 774/1000
305/305 [==============================] - 0s 177us/step - loss: 0.1971 - accuracy: 0.8852 - val_loss: 0.2053 - val_accuracy: 0.8788
Epoch 775/1000
305/305 [==============================] - 0s 102us/step - loss: 0.1981 - accuracy: 0.8852 - val_loss: 0.2104 - val_accuracy: 0.8864
Epoch 776/1000
305/305 [==============================] - 0s 108us/step - loss: 0.1990 - accuracy: 0.8852 - val_loss: 0.2057 - val_accuracy: 0.8788
Epoch 777/1000
305/305 [==============================] - 0s 105us/step - loss: 0.1970 - accuracy: 0.8852 - val_loss: 0.2053 - val_accuracy: 0.8788
Epoch 778/1000
305/305 [==============================] - 0s 113us/step - loss: 0.1968 - accuracy: 0.8852 - val_loss: 0.2053 - 

305/305 [==============================] - 0s 87us/step - loss: 0.1967 - accuracy: 0.8852 - val_loss: 0.2055 - val_accuracy: 0.8864
Epoch 828/1000
305/305 [==============================] - 0s 164us/step - loss: 0.1968 - accuracy: 0.8852 - val_loss: 0.2049 - val_accuracy: 0.8788
Epoch 829/1000
305/305 [==============================] - 0s 127us/step - loss: 0.1962 - accuracy: 0.8852 - val_loss: 0.2047 - val_accuracy: 0.8788
Epoch 830/1000
305/305 [==============================] - 0s 138us/step - loss: 0.1961 - accuracy: 0.8852 - val_loss: 0.2060 - val_accuracy: 0.8864
Epoch 831/1000
305/305 [==============================] - 0s 120us/step - loss: 0.1970 - accuracy: 0.8852 - val_loss: 0.2047 - val_accuracy: 0.8864
Epoch 832/1000
305/305 [==============================] - 0s 136us/step - loss: 0.1963 - accuracy: 0.8852 - val_loss: 0.2046 - val_accuracy: 0.8788
Epoch 833/1000
305/305 [==============================] - 0s 122us/step - loss: 0.1960 - accuracy: 0.8852 - val_loss: 0.2041 - v

305/305 [==============================] - 0s 111us/step - loss: 0.1963 - accuracy: 0.8852 - val_loss: 0.2046 - val_accuracy: 0.8864
Epoch 883/1000
305/305 [==============================] - 0s 118us/step - loss: 0.1966 - accuracy: 0.8852 - val_loss: 0.2040 - val_accuracy: 0.8864
Epoch 884/1000
305/305 [==============================] - 0s 99us/step - loss: 0.1961 - accuracy: 0.8852 - val_loss: 0.2039 - val_accuracy: 0.8864
Epoch 885/1000
305/305 [==============================] - 0s 105us/step - loss: 0.1960 - accuracy: 0.8852 - val_loss: 0.2035 - val_accuracy: 0.8864
Epoch 886/1000
305/305 [==============================] - 0s 107us/step - loss: 0.1959 - accuracy: 0.8852 - val_loss: 0.2049 - val_accuracy: 0.8864
Epoch 887/1000
305/305 [==============================] - 0s 88us/step - loss: 0.1967 - accuracy: 0.8852 - val_loss: 0.2039 - val_accuracy: 0.8864
Epoch 888/1000
305/305 [==============================] - 0s 98us/step - loss: 0.1961 - accuracy: 0.8852 - val_loss: 0.2038 - val

305/305 [==============================] - 0s 106us/step - loss: 0.1959 - accuracy: 0.8852 - val_loss: 0.2038 - val_accuracy: 0.8864
Epoch 938/1000
305/305 [==============================] - 0s 77us/step - loss: 0.1957 - accuracy: 0.8852 - val_loss: 0.2036 - val_accuracy: 0.8864
Epoch 939/1000
305/305 [==============================] - 0s 96us/step - loss: 0.1957 - accuracy: 0.8852 - val_loss: 0.2028 - val_accuracy: 0.8864
Epoch 940/1000
305/305 [==============================] - 0s 132us/step - loss: 0.2110 - accuracy: 0.8852 - val_loss: 0.2066 - val_accuracy: 0.8864
Epoch 941/1000
305/305 [==============================] - 0s 108us/step - loss: 0.1972 - accuracy: 0.8852 - val_loss: 0.2044 - val_accuracy: 0.8788
Epoch 942/1000
305/305 [==============================] - 0s 134us/step - loss: 0.1960 - accuracy: 0.8852 - val_loss: 0.2041 - val_accuracy: 0.8788
Epoch 943/1000
305/305 [==============================] - 0s 129us/step - loss: 0.1958 - accuracy: 0.8852 - val_loss: 0.2039 - va

305/305 [==============================] - 0s 121us/step - loss: 0.1966 - accuracy: 0.8852 - val_loss: 0.2035 - val_accuracy: 0.8864
Epoch 993/1000
305/305 [==============================] - 0s 97us/step - loss: 0.1956 - accuracy: 0.8852 - val_loss: 0.2033 - val_accuracy: 0.8864
Epoch 994/1000
305/305 [==============================] - 0s 84us/step - loss: 0.1955 - accuracy: 0.8852 - val_loss: 0.2032 - val_accuracy: 0.8864
Epoch 995/1000
305/305 [==============================] - 0s 130us/step - loss: 0.1954 - accuracy: 0.8852 - val_loss: 0.2021 - val_accuracy: 0.8864
Epoch 996/1000
305/305 [==============================] - 0s 100us/step - loss: 0.2206 - accuracy: 0.8852 - val_loss: 0.2390 - val_accuracy: 0.8864
Epoch 997/1000
305/305 [==============================] - 0s 103us/step - loss: 0.2045 - accuracy: 0.8852 - val_loss: 0.2050 - val_accuracy: 0.8788
Epoch 998/1000
305/305 [==============================] - 0s 119us/step - loss: 0.1960 - accuracy: 0.8852 - val_loss: 0.2044 - va

In [23]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

132/132 [==============================] - 0s 103us/step
combination test accuracy: 87.88%


In [24]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [25]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 336 samples, validate on 144 samples
Epoch 1/1000
336/336 [==============================] - 0s 463us/step - loss: 3.5930 - accuracy: 0.5893 - val_loss: 0.6268 - val_accuracy: 0.7014
Epoch 2/1000
336/336 [==============================] - 0s 94us/step - loss: 0.6378 - accuracy: 0.6726 - val_loss: 0.6289 - val_accuracy: 0.6319
Epoch 3/1000
336/336 [==============================] - 0s 94us/step - loss: 0.6285 - accuracy: 0.6964 - val_loss: 0.6012 - val_accuracy: 0.6806
Epoch 4/1000
336/336 [==============================] - 0s 116us/step - loss: 0.5968 - accuracy: 0.6964 - val_loss: 0.6410 - val_accuracy: 0.6319
Epoch 5/1000
336/336 [==============================] - 0s 126us/step - loss: 0.6072 - accuracy: 0.6905 - val_loss: 0.5847 - val_accuracy: 0.6875
Epoch 6/1000
336/336 [==============================] - 0s 119us/step - loss: 0.5804 - accuracy: 0.7262 - val_loss: 0.5773 - val_accuracy: 0.7222
Epoch 7/1000
336/336 [==============================] - 0s 114us/step - loss: 0.

336/336 [==============================] - 0s 109us/step - loss: 0.3740 - accuracy: 0.8452 - val_loss: 0.3949 - val_accuracy: 0.8403
Epoch 57/1000
336/336 [==============================] - 0s 92us/step - loss: 0.3702 - accuracy: 0.8482 - val_loss: 0.3909 - val_accuracy: 0.8472
Epoch 58/1000
336/336 [==============================] - 0s 80us/step - loss: 0.3719 - accuracy: 0.8452 - val_loss: 0.3881 - val_accuracy: 0.8403
Epoch 59/1000
336/336 [==============================] - 0s 82us/step - loss: 0.3675 - accuracy: 0.8452 - val_loss: 0.3878 - val_accuracy: 0.8264
Epoch 60/1000
336/336 [==============================] - 0s 92us/step - loss: 0.3687 - accuracy: 0.8452 - val_loss: 0.3854 - val_accuracy: 0.8403
Epoch 61/1000
336/336 [==============================] - 0s 97us/step - loss: 0.3671 - accuracy: 0.8482 - val_loss: 0.3846 - val_accuracy: 0.8333
Epoch 62/1000
336/336 [==============================] - 0s 84us/step - loss: 0.3619 - accuracy: 0.8512 - val_loss: 0.3905 - val_accuracy

Epoch 112/1000
336/336 [==============================] - 0s 136us/step - loss: 0.3353 - accuracy: 0.8512 - val_loss: 0.3459 - val_accuracy: 0.8403
Epoch 113/1000
336/336 [==============================] - 0s 89us/step - loss: 0.3274 - accuracy: 0.8542 - val_loss: 0.3437 - val_accuracy: 0.8403
Epoch 114/1000
336/336 [==============================] - 0s 87us/step - loss: 0.3304 - accuracy: 0.8512 - val_loss: 0.3462 - val_accuracy: 0.8403
Epoch 115/1000
336/336 [==============================] - 0s 92us/step - loss: 0.3294 - accuracy: 0.8512 - val_loss: 0.3587 - val_accuracy: 0.8264
Epoch 116/1000
336/336 [==============================] - 0s 104us/step - loss: 0.3281 - accuracy: 0.8482 - val_loss: 0.3425 - val_accuracy: 0.8403
Epoch 117/1000
336/336 [==============================] - 0s 100us/step - loss: 0.3204 - accuracy: 0.8542 - val_loss: 0.3697 - val_accuracy: 0.8542
Epoch 118/1000
336/336 [==============================] - 0s 100us/step - loss: 0.3264 - accuracy: 0.8542 - val_los

336/336 [==============================] - 0s 91us/step - loss: 0.3206 - accuracy: 0.8601 - val_loss: 0.3267 - val_accuracy: 0.8403
Epoch 168/1000
336/336 [==============================] - 0s 85us/step - loss: 0.3117 - accuracy: 0.8601 - val_loss: 0.3262 - val_accuracy: 0.8472
Epoch 169/1000
336/336 [==============================] - 0s 87us/step - loss: 0.3118 - accuracy: 0.8601 - val_loss: 0.3303 - val_accuracy: 0.8403
Epoch 170/1000
336/336 [==============================] - 0s 98us/step - loss: 0.3097 - accuracy: 0.8601 - val_loss: 0.4230 - val_accuracy: 0.7708
Epoch 171/1000
336/336 [==============================] - 0s 87us/step - loss: 0.3172 - accuracy: 0.8601 - val_loss: 0.3504 - val_accuracy: 0.8333
Epoch 172/1000
336/336 [==============================] - 0s 104us/step - loss: 0.3157 - accuracy: 0.8601 - val_loss: 0.3294 - val_accuracy: 0.8542
Epoch 173/1000
336/336 [==============================] - 0s 105us/step - loss: 0.3071 - accuracy: 0.8631 - val_loss: 0.3277 - val_a

336/336 [==============================] - 0s 135us/step - loss: 0.3024 - accuracy: 0.8631 - val_loss: 0.3192 - val_accuracy: 0.8403
Epoch 223/1000
336/336 [==============================] - 0s 141us/step - loss: 0.2994 - accuracy: 0.8631 - val_loss: 0.3175 - val_accuracy: 0.8472
Epoch 224/1000
336/336 [==============================] - 0s 183us/step - loss: 0.2993 - accuracy: 0.8601 - val_loss: 0.3204 - val_accuracy: 0.8403
Epoch 225/1000
336/336 [==============================] - 0s 126us/step - loss: 0.2935 - accuracy: 0.8631 - val_loss: 0.3229 - val_accuracy: 0.8403
Epoch 226/1000
336/336 [==============================] - 0s 161us/step - loss: 0.3020 - accuracy: 0.8601 - val_loss: 0.3214 - val_accuracy: 0.8403
Epoch 227/1000
336/336 [==============================] - 0s 158us/step - loss: 0.3113 - accuracy: 0.8482 - val_loss: 0.3211 - val_accuracy: 0.8403
Epoch 228/1000
336/336 [==============================] - 0s 117us/step - loss: 0.2991 - accuracy: 0.8631 - val_loss: 0.3196 - 

...

Epoch 775/1000
336/336 [==============================] - 0s 107us/step - loss: 0.2975 - accuracy: 0.8512 - val_loss: 0.3190 - val_accuracy: 0.8403
Epoch 776/1000
336/336 [==============================] - 0s 80us/step - loss: 0.2929 - accuracy: 0.8571 - val_loss: 0.3196 - val_accuracy: 0.8333
Epoch 777/1000
336/336 [==============================] - 0s 93us/step - loss: 0.3030 - accuracy: 0.8571 - val_loss: 0.3168 - val_accuracy: 0.8472
Epoch 778/1000
336/336 [==============================] - 0s 96us/step - loss: 0.2957 - accuracy: 0.8571 - val_loss: 0.3213 - val_accuracy: 0.8333
Epoch 779/1000
336/336 [==============================] - 0s 82us/step - loss: 0.3005 - accuracy: 0.8482 - val_loss: 0.3238 - val_accuracy: 0.8542
Epoch 780/1000
336/336 [==============================] - 0s 68us/step - loss: 0.2956 - accuracy: 0.8571 - val_loss: 0.3203 - val_accuracy: 0.8403
Epoch 781/1000
336/336 [==============================] - 0s 76us/step - loss: 0.2952 - accuracy: 0.8601 - val_loss: 

336/336 [==============================] - 0s 178us/step - loss: 0.2880 - accuracy: 0.8601 - val_loss: 0.3465 - val_accuracy: 0.8194
Epoch 831/1000
336/336 [==============================] - 0s 125us/step - loss: 0.2869 - accuracy: 0.8631 - val_loss: 0.3408 - val_accuracy: 0.8611
Epoch 832/1000
336/336 [==============================] - 0s 166us/step - loss: 0.2978 - accuracy: 0.8482 - val_loss: 0.3198 - val_accuracy: 0.8403
Epoch 833/1000
336/336 [==============================] - 0s 136us/step - loss: 0.2889 - accuracy: 0.8571 - val_loss: 0.3062 - val_accuracy: 0.8472
Epoch 834/1000
336/336 [==============================] - 0s 153us/step - loss: 0.2871 - accuracy: 0.8631 - val_loss: 0.3165 - val_accuracy: 0.8472
Epoch 835/1000
336/336 [==============================] - 0s 124us/step - loss: 0.2900 - accuracy: 0.8601 - val_loss: 0.3444 - val_accuracy: 0.8611
Epoch 836/1000
336/336 [==============================] - 0s 209us/step - loss: 0.2852 - accuracy: 0.8631 - val_loss: 0.3459 - 

336/336 [==============================] - 0s 100us/step - loss: 0.2790 - accuracy: 0.8661 - val_loss: 0.3101 - val_accuracy: 0.8472
Epoch 886/1000
336/336 [==============================] - 0s 92us/step - loss: 0.2868 - accuracy: 0.8601 - val_loss: 0.3593 - val_accuracy: 0.7778
Epoch 887/1000
336/336 [==============================] - 0s 98us/step - loss: 0.2895 - accuracy: 0.8512 - val_loss: 0.3104 - val_accuracy: 0.8472
Epoch 888/1000
336/336 [==============================] - 0s 83us/step - loss: 0.2934 - accuracy: 0.8482 - val_loss: 0.3129 - val_accuracy: 0.8472
Epoch 889/1000
336/336 [==============================] - 0s 98us/step - loss: 0.2853 - accuracy: 0.8631 - val_loss: 0.3063 - val_accuracy: 0.8472
Epoch 890/1000
336/336 [==============================] - 0s 89us/step - loss: 0.2898 - accuracy: 0.8601 - val_loss: 0.3060 - val_accuracy: 0.8472
Epoch 891/1000
336/336 [==============================] - 0s 83us/step - loss: 0.2815 - accuracy: 0.8601 - val_loss: 0.3156 - val_ac

336/336 [==============================] - 0s 177us/step - loss: 0.2797 - accuracy: 0.8631 - val_loss: 0.3115 - val_accuracy: 0.8542
Epoch 941/1000
336/336 [==============================] - 0s 120us/step - loss: 0.2897 - accuracy: 0.8601 - val_loss: 0.3066 - val_accuracy: 0.8472
Epoch 942/1000
336/336 [==============================] - 0s 102us/step - loss: 0.2900 - accuracy: 0.8571 - val_loss: 0.3066 - val_accuracy: 0.8472
Epoch 943/1000
336/336 [==============================] - 0s 100us/step - loss: 0.2810 - accuracy: 0.8601 - val_loss: 0.3056 - val_accuracy: 0.8472
Epoch 944/1000
336/336 [==============================] - 0s 93us/step - loss: 0.2854 - accuracy: 0.8542 - val_loss: 0.3153 - val_accuracy: 0.8472
Epoch 945/1000
336/336 [==============================] - 0s 95us/step - loss: 0.2782 - accuracy: 0.8601 - val_loss: 0.3963 - val_accuracy: 0.7639
Epoch 946/1000
336/336 [==============================] - 0s 88us/step - loss: 0.2792 - accuracy: 0.8631 - val_loss: 0.3058 - val

336/336 [==============================] - 0s 103us/step - loss: 0.2803 - accuracy: 0.8601 - val_loss: 0.3023 - val_accuracy: 0.8472
Epoch 996/1000
336/336 [==============================] - 0s 88us/step - loss: 0.2770 - accuracy: 0.8661 - val_loss: 0.3065 - val_accuracy: 0.8472
Epoch 997/1000
336/336 [==============================] - 0s 90us/step - loss: 0.2775 - accuracy: 0.8631 - val_loss: 0.3251 - val_accuracy: 0.8264
Epoch 998/1000
336/336 [==============================] - 0s 82us/step - loss: 0.2830 - accuracy: 0.8542 - val_loss: 0.3224 - val_accuracy: 0.8542
Epoch 999/1000
336/336 [==============================] - 0s 79us/step - loss: 0.2872 - accuracy: 0.8631 - val_loss: 0.3051 - val_accuracy: 0.8472
Epoch 1000/1000
336/336 [==============================] - 0s 118us/step - loss: 0.2795 - accuracy: 0.8631 - val_loss: 0.3057 - val_accuracy: 0.8472


In [27]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

144/144 [==============================] - 0s 85us/step
over-sampling test accuracy: 84.72%


In [28]:
############ Feature selection using lasso ##########

In [29]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [30]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [31]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [32]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [33]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 866
selected features: 43


In [34]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  4,  12,  29,  55,  91, 129, 145, 157, 175, 276, 278, 322, 353,
        356, 358, 362, 371, 415, 416, 490, 532, 541, 549, 552, 571, 600,
        644, 645, 655, 672, 673, 685, 689, 694, 695, 697, 709, 711, 726,
        729, 741, 864, 865]])

In [35]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA',
       'TTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAAT',
       'TTTTATAGTACTTTTTTAAACTTTGCTATATCTCTTGTGTTATGA',
       'TTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAATCA',
       'TTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAATCATATTATTT',
       'TTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAATCAT',
       'TTATCGACAGAT', 'TTACTGACAAG', 'TTAACATAATAAT',
       'TCATAACACAAGAGATATAGCAAAGTTTAAAAAAGTACTATAAAATAGCAATTGAATGTCCAGTAACAAATTTGGAGGAAGCGTATATGTATCAAACAAT',
       'TCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAATCATA',
       'TATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAATGTCATAGTGATATTTAAGGGAGATTG',
 

In [36]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([  0,   1,   2,   3,   5,   6,   7,   8,   9,  10,
            ...
            854, 855, 856, 857, 858, 859, 860, 861, 862, 863],
           dtype='int64', length=823)

In [37]:
len(removed_features)

823

In [38]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 43) (255,) (255, 44)


In [39]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 209), (1, 213)]


In [40]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 240), (1, 240)]


In [41]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [42]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [43]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [44]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [45]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 295 samples, validate on 127 samples
Epoch 1/1000
295/295 [==============================] - 0s 497us/step - loss: 12.0907 - accuracy: 0.6068 - val_loss: 0.7226 - val_accuracy: 0.6614
Epoch 2/1000
295/295 [==============================] - 0s 107us/step - loss: 0.6396 - accuracy: 0.6373 - val_loss: 0.5657 - val_accuracy: 0.8110
Epoch 3/1000
295/295 [==============================] - 0s 81us/step - loss: 0.6024 - accuracy: 0.7017 - val_loss: 0.5400 - val_accuracy: 0.8504
Epoch 4/1000
295/295 [==============================] - 0s 96us/step - loss: 0.5813 - accuracy: 0.7390 - val_loss: 0.5564 - val_accuracy: 0.7402
Epoch 5/1000
295/295 [==============================] - 0s 123us/step - loss: 0.5730 - accuracy: 0.7322 - val_loss: 0.5876 - val_accuracy: 0.7244
Epoch 6/1000
295/295 [==============================] - 0s 97us/step - loss: 0.5778 - accuracy: 0.7288 - val_loss: 0.5091 - val_accuracy: 0.8425
Epoch 7/1000
295/295 [==============================] - 0s 147us/step - loss: 0.

295/295 [==============================] - 0s 100us/step - loss: 0.4457 - accuracy: 0.8237 - val_loss: 0.3630 - val_accuracy: 0.8661
Epoch 57/1000
295/295 [==============================] - 0s 85us/step - loss: 0.4317 - accuracy: 0.8339 - val_loss: 0.3591 - val_accuracy: 0.8661
Epoch 58/1000
295/295 [==============================] - 0s 82us/step - loss: 0.4352 - accuracy: 0.8203 - val_loss: 0.3579 - val_accuracy: 0.8661
Epoch 59/1000
295/295 [==============================] - 0s 88us/step - loss: 0.4302 - accuracy: 0.8237 - val_loss: 0.3594 - val_accuracy: 0.8661
Epoch 60/1000
295/295 [==============================] - 0s 87us/step - loss: 0.4125 - accuracy: 0.8373 - val_loss: 0.3496 - val_accuracy: 0.8661
Epoch 61/1000
295/295 [==============================] - 0s 87us/step - loss: 0.4137 - accuracy: 0.8373 - val_loss: 0.3467 - val_accuracy: 0.8661
Epoch 62/1000
295/295 [==============================] - 0s 83us/step - loss: 0.4226 - accuracy: 0.8169 - val_loss: 0.3468 - val_accuracy

Epoch 112/1000
295/295 [==============================] - 0s 71us/step - loss: 0.3878 - accuracy: 0.8407 - val_loss: 0.3348 - val_accuracy: 0.8583
Epoch 113/1000
295/295 [==============================] - 0s 82us/step - loss: 0.3948 - accuracy: 0.8339 - val_loss: 0.3311 - val_accuracy: 0.8819
Epoch 114/1000
295/295 [==============================] - 0s 92us/step - loss: 0.3892 - accuracy: 0.8407 - val_loss: 0.3323 - val_accuracy: 0.8819
Epoch 115/1000
295/295 [==============================] - 0s 90us/step - loss: 0.3813 - accuracy: 0.8441 - val_loss: 0.3345 - val_accuracy: 0.8819
Epoch 116/1000
295/295 [==============================] - 0s 88us/step - loss: 0.3795 - accuracy: 0.8475 - val_loss: 0.3343 - val_accuracy: 0.8819
Epoch 117/1000
295/295 [==============================] - 0s 87us/step - loss: 0.3903 - accuracy: 0.8441 - val_loss: 0.3837 - val_accuracy: 0.8583
Epoch 118/1000
295/295 [==============================] - 0s 91us/step - loss: 0.4004 - accuracy: 0.8203 - val_loss: 0

295/295 [==============================] - 0s 116us/step - loss: 0.3611 - accuracy: 0.8407 - val_loss: 0.3128 - val_accuracy: 0.8583
Epoch 168/1000
295/295 [==============================] - 0s 93us/step - loss: 0.3941 - accuracy: 0.8169 - val_loss: 0.3047 - val_accuracy: 0.8583
Epoch 169/1000
295/295 [==============================] - 0s 86us/step - loss: 0.3598 - accuracy: 0.8305 - val_loss: 0.3206 - val_accuracy: 0.8583
Epoch 170/1000
295/295 [==============================] - 0s 92us/step - loss: 0.3499 - accuracy: 0.8373 - val_loss: 1.0342 - val_accuracy: 0.7165
Epoch 171/1000
295/295 [==============================] - 0s 96us/step - loss: 0.4344 - accuracy: 0.8068 - val_loss: 0.3090 - val_accuracy: 0.8583
Epoch 172/1000
295/295 [==============================] - 0s 89us/step - loss: 0.3414 - accuracy: 0.8407 - val_loss: 0.3258 - val_accuracy: 0.8583
Epoch 173/1000
295/295 [==============================] - 0s 99us/step - loss: 0.3580 - accuracy: 0.8271 - val_loss: 0.4693 - val_ac

295/295 [==============================] - 0s 182us/step - loss: 0.3127 - accuracy: 0.8508 - val_loss: 0.2764 - val_accuracy: 0.8819
Epoch 223/1000
295/295 [==============================] - 0s 168us/step - loss: 0.3167 - accuracy: 0.8542 - val_loss: 0.2787 - val_accuracy: 0.8819
Epoch 224/1000
295/295 [==============================] - 0s 199us/step - loss: 0.3124 - accuracy: 0.8542 - val_loss: 0.2852 - val_accuracy: 0.8819
Epoch 225/1000
295/295 [==============================] - 0s 462us/step - loss: 0.7370 - accuracy: 0.8136 - val_loss: 0.3461 - val_accuracy: 0.8661
Epoch 226/1000
295/295 [==============================] - 0s 133us/step - loss: 0.3510 - accuracy: 0.8712 - val_loss: 0.3150 - val_accuracy: 0.8661
Epoch 227/1000
295/295 [==============================] - 0s 101us/step - loss: 0.3596 - accuracy: 0.8475 - val_loss: 3.1146 - val_accuracy: 0.5118
Epoch 228/1000
295/295 [==============================] - 0s 110us/step - loss: 0.7956 - accuracy: 0.7627 - val_loss: 0.4221 - 

...

Epoch 775/1000
295/295 [==============================] - 0s 92us/step - loss: 0.4538 - accuracy: 0.7627 - val_loss: 0.3648 - val_accuracy: 0.8504
Epoch 776/1000
295/295 [==============================] - 0s 82us/step - loss: 0.4532 - accuracy: 0.7627 - val_loss: 0.3646 - val_accuracy: 0.8504
Epoch 777/1000
295/295 [==============================] - 0s 84us/step - loss: 0.4540 - accuracy: 0.7627 - val_loss: 0.3650 - val_accuracy: 0.8504
Epoch 778/1000
295/295 [==============================] - 0s 81us/step - loss: 0.4540 - accuracy: 0.7627 - val_loss: 0.3651 - val_accuracy: 0.8504
Epoch 779/1000
295/295 [==============================] - 0s 83us/step - loss: 0.4532 - accuracy: 0.7627 - val_loss: 0.3649 - val_accuracy: 0.8504
Epoch 780/1000
295/295 [==============================] - 0s 83us/step - loss: 0.4532 - accuracy: 0.7627 - val_loss: 0.3648 - val_accuracy: 0.8504
Epoch 781/1000
295/295 [==============================] - 0s 114us/step - loss: 0.4532 - accuracy: 0.7627 - val_loss: 

295/295 [==============================] - 0s 97us/step - loss: 0.3361 - accuracy: 0.8508 - val_loss: 0.2838 - val_accuracy: 0.8898
Epoch 831/1000
295/295 [==============================] - 0s 95us/step - loss: 0.3192 - accuracy: 0.8610 - val_loss: 0.2749 - val_accuracy: 0.8898
Epoch 832/1000
295/295 [==============================] - 0s 103us/step - loss: 0.3449 - accuracy: 0.8508 - val_loss: 0.3585 - val_accuracy: 0.8268
Epoch 833/1000
295/295 [==============================] - 0s 97us/step - loss: 0.6003 - accuracy: 0.8203 - val_loss: 0.3372 - val_accuracy: 0.8740
Epoch 834/1000
295/295 [==============================] - 0s 81us/step - loss: 0.3468 - accuracy: 0.8610 - val_loss: 0.3361 - val_accuracy: 0.8740
Epoch 835/1000
295/295 [==============================] - 0s 76us/step - loss: 0.3416 - accuracy: 0.8678 - val_loss: 0.3336 - val_accuracy: 0.8661
Epoch 836/1000
295/295 [==============================] - 0s 87us/step - loss: 0.3379 - accuracy: 0.8712 - val_loss: 0.3286 - val_ac

Epoch 886/1000
295/295 [==============================] - 0s 92us/step - loss: 0.2830 - accuracy: 0.8746 - val_loss: 0.2541 - val_accuracy: 0.8976
Epoch 887/1000
295/295 [==============================] - 0s 82us/step - loss: 0.2820 - accuracy: 0.8746 - val_loss: 0.2530 - val_accuracy: 0.8976
Epoch 888/1000
295/295 [==============================] - 0s 96us/step - loss: 0.3061 - accuracy: 0.8644 - val_loss: 0.4402 - val_accuracy: 0.8740
Epoch 889/1000
295/295 [==============================] - 0s 103us/step - loss: 0.4874 - accuracy: 0.8407 - val_loss: 0.2584 - val_accuracy: 0.8898
Epoch 890/1000
295/295 [==============================] - 0s 109us/step - loss: 0.2846 - accuracy: 0.8712 - val_loss: 0.2590 - val_accuracy: 0.8898
Epoch 891/1000
295/295 [==============================] - 0s 98us/step - loss: 0.2870 - accuracy: 0.8780 - val_loss: 0.2552 - val_accuracy: 0.8898
Epoch 892/1000
295/295 [==============================] - 0s 107us/step - loss: 0.2938 - accuracy: 0.8678 - val_loss

295/295 [==============================] - 0s 91us/step - loss: 0.2814 - accuracy: 0.8542 - val_loss: 0.2297 - val_accuracy: 0.9055
Epoch 942/1000
295/295 [==============================] - 0s 55us/step - loss: 0.2544 - accuracy: 0.8780 - val_loss: 6.9306 - val_accuracy: 0.5354
Epoch 943/1000
295/295 [==============================] - 0s 99us/step - loss: 1.5970 - accuracy: 0.6949 - val_loss: 0.3925 - val_accuracy: 0.8425
Epoch 944/1000
295/295 [==============================] - 0s 56us/step - loss: 0.5969 - accuracy: 0.7525 - val_loss: 0.3650 - val_accuracy: 0.8504
Epoch 945/1000
295/295 [==============================] - 0s 107us/step - loss: 0.4915 - accuracy: 0.7525 - val_loss: 0.3631 - val_accuracy: 0.8504
Epoch 946/1000
295/295 [==============================] - 0s 61us/step - loss: 0.4691 - accuracy: 0.7525 - val_loss: 0.3655 - val_accuracy: 0.8504
Epoch 947/1000
295/295 [==============================] - 0s 88us/step - loss: 0.4555 - accuracy: 0.7559 - val_loss: 0.3684 - val_ac

Epoch 997/1000
295/295 [==============================] - 0s 71us/step - loss: 0.4202 - accuracy: 0.8237 - val_loss: 0.3689 - val_accuracy: 0.8346
Epoch 998/1000
295/295 [==============================] - 0s 70us/step - loss: 0.4195 - accuracy: 0.7966 - val_loss: 0.3571 - val_accuracy: 0.8661
Epoch 999/1000
295/295 [==============================] - 0s 68us/step - loss: 0.4189 - accuracy: 0.8203 - val_loss: 0.3598 - val_accuracy: 0.8661
Epoch 1000/1000
295/295 [==============================] - 0s 62us/step - loss: 0.4179 - accuracy: 0.8339 - val_loss: 0.3751 - val_accuracy: 0.8346


In [46]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


127/127 [==============================] - 0s 56us/step
combination test accuracy: 83.46%


In [65]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [66]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [67]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 336 samples, validate on 144 samples
Epoch 1/1000
336/336 [==============================] - 0s 531us/step - loss: 12.1291 - accuracy: 0.4494 - val_loss: 0.7014 - val_accuracy: 0.3611
Epoch 2/1000
336/336 [==============================] - 0s 175us/step - loss: 0.6990 - accuracy: 0.3512 - val_loss: 0.7000 - val_accuracy: 0.3611
Epoch 3/1000
336/336 [==============================] - 0s 93us/step - loss: 0.6976 - accuracy: 0.3274 - val_loss: 0.6986 - val_accuracy: 0.3611
Epoch 4/1000
336/336 [==============================] - 0s 92us/step - loss: 0.6960 - accuracy: 0.3750 - val_loss: 0.6974 - val_accuracy: 0.4236
Epoch 5/1000
336/336 [==============================] - 0s 94us/step - loss: 0.6948 - accuracy: 0.4256 - val_loss: 0.6963 - val_accuracy: 0.4931
Epoch 6/1000
336/336 [==============================] - 0s 85us/step - loss: 0.6936 - accuracy: 0.4673 - val_loss: 0.6952 - val_accuracy: 0.4722
Epoch 7/1000
336/336 [==============================] - 0s 97us/step - loss: 0.69

336/336 [==============================] - 0s 98us/step - loss: 0.5623 - accuracy: 0.7708 - val_loss: 0.5844 - val_accuracy: 0.7222
Epoch 57/1000
336/336 [==============================] - 0s 85us/step - loss: 0.5603 - accuracy: 0.7619 - val_loss: 0.5915 - val_accuracy: 0.7361
Epoch 58/1000
336/336 [==============================] - 0s 85us/step - loss: 0.5580 - accuracy: 0.7708 - val_loss: 0.5814 - val_accuracy: 0.7361
Epoch 59/1000
336/336 [==============================] - 0s 79us/step - loss: 0.5608 - accuracy: 0.7500 - val_loss: 0.6106 - val_accuracy: 0.6875
Epoch 60/1000
336/336 [==============================] - 0s 71us/step - loss: 0.5652 - accuracy: 0.7411 - val_loss: 0.7428 - val_accuracy: 0.6250
Epoch 61/1000
336/336 [==============================] - 0s 91us/step - loss: 0.5728 - accuracy: 0.7470 - val_loss: 0.5774 - val_accuracy: 0.7292
Epoch 62/1000
336/336 [==============================] - 0s 67us/step - loss: 0.5485 - accuracy: 0.7768 - val_loss: 0.6018 - val_accuracy:

Epoch 112/1000
336/336 [==============================] - 0s 175us/step - loss: 0.5098 - accuracy: 0.7619 - val_loss: 0.5519 - val_accuracy: 0.6875
Epoch 113/1000
336/336 [==============================] - 0s 142us/step - loss: 0.5032 - accuracy: 0.7679 - val_loss: 0.5376 - val_accuracy: 0.7292
Epoch 114/1000
336/336 [==============================] - 0s 137us/step - loss: 0.4967 - accuracy: 0.7768 - val_loss: 0.5314 - val_accuracy: 0.7292
Epoch 115/1000
336/336 [==============================] - 0s 115us/step - loss: 0.4921 - accuracy: 0.7768 - val_loss: 0.5295 - val_accuracy: 0.7292
Epoch 116/1000
336/336 [==============================] - 0s 121us/step - loss: 0.4931 - accuracy: 0.7768 - val_loss: 0.5415 - val_accuracy: 0.7014
Epoch 117/1000
336/336 [==============================] - 0s 121us/step - loss: 0.4950 - accuracy: 0.7798 - val_loss: 0.5841 - val_accuracy: 0.6458
Epoch 118/1000
336/336 [==============================] - 0s 100us/step - loss: 0.4946 - accuracy: 0.7679 - val_

336/336 [==============================] - 0s 356us/step - loss: 0.4501 - accuracy: 0.8036 - val_loss: 0.4974 - val_accuracy: 0.7639
Epoch 168/1000
336/336 [==============================] - 0s 187us/step - loss: 0.4396 - accuracy: 0.8065 - val_loss: 0.4910 - val_accuracy: 0.7639
Epoch 169/1000
336/336 [==============================] - 0s 162us/step - loss: 0.4430 - accuracy: 0.7976 - val_loss: 0.5223 - val_accuracy: 0.7222
Epoch 170/1000
336/336 [==============================] - 0s 221us/step - loss: 0.4426 - accuracy: 0.8065 - val_loss: 0.5069 - val_accuracy: 0.7639
Epoch 171/1000
336/336 [==============================] - 0s 137us/step - loss: 0.4418 - accuracy: 0.8006 - val_loss: 0.4885 - val_accuracy: 0.7639
Epoch 172/1000
336/336 [==============================] - 0s 153us/step - loss: 0.4354 - accuracy: 0.8095 - val_loss: 0.5034 - val_accuracy: 0.7431
Epoch 173/1000
336/336 [==============================] - 0s 179us/step - loss: 0.4396 - accuracy: 0.7917 - val_loss: 0.4859 - 

336/336 [==============================] - 0s 161us/step - loss: 0.4346 - accuracy: 0.7827 - val_loss: 0.6566 - val_accuracy: 0.6806
Epoch 223/1000
336/336 [==============================] - 0s 197us/step - loss: 0.4130 - accuracy: 0.8036 - val_loss: 0.4468 - val_accuracy: 0.8056
Epoch 224/1000
336/336 [==============================] - 0s 343us/step - loss: 0.3992 - accuracy: 0.8125 - val_loss: 0.4932 - val_accuracy: 0.7222
Epoch 225/1000
336/336 [==============================] - 0s 145us/step - loss: 0.4025 - accuracy: 0.7917 - val_loss: 0.4566 - val_accuracy: 0.8056
Epoch 226/1000
336/336 [==============================] - 0s 156us/step - loss: 0.4009 - accuracy: 0.8214 - val_loss: 0.4461 - val_accuracy: 0.7639
Epoch 227/1000
336/336 [==============================] - 0s 300us/step - loss: 0.3934 - accuracy: 0.8065 - val_loss: 0.4456 - val_accuracy: 0.7639
Epoch 228/1000
336/336 [==============================] - 0s 167us/step - loss: 0.3881 - accuracy: 0.8095 - val_loss: 0.4510 - 

...

336/336 [==============================] - 0s 148us/step - loss: 0.3138 - accuracy: 0.8423 - val_loss: 0.3489 - val_accuracy: 0.8264
Epoch 773/1000
336/336 [==============================] - 0s 118us/step - loss: 0.3337 - accuracy: 0.8274 - val_loss: 0.3479 - val_accuracy: 0.8264
Epoch 774/1000
336/336 [==============================] - 0s 106us/step - loss: 0.3113 - accuracy: 0.8423 - val_loss: 0.3932 - val_accuracy: 0.7917
Epoch 775/1000
336/336 [==============================] - 0s 141us/step - loss: 0.3160 - accuracy: 0.8393 - val_loss: 0.3528 - val_accuracy: 0.8264
Epoch 776/1000
336/336 [==============================] - 0s 130us/step - loss: 0.3259 - accuracy: 0.8393 - val_loss: 0.3467 - val_accuracy: 0.8264
Epoch 777/1000
336/336 [==============================] - 0s 122us/step - loss: 0.3138 - accuracy: 0.8423 - val_loss: 0.3479 - val_accuracy: 0.8264
Epoch 778/1000
336/336 [==============================] - 0s 212us/step - loss: 0.3108 - accuracy: 0.8393 - val_loss: 0.3507 - 

336/336 [==============================] - 0s 120us/step - loss: 0.3142 - accuracy: 0.8393 - val_loss: 0.3460 - val_accuracy: 0.8333
Epoch 828/1000
336/336 [==============================] - 0s 177us/step - loss: 0.3136 - accuracy: 0.8423 - val_loss: 0.3460 - val_accuracy: 0.8333
Epoch 829/1000
336/336 [==============================] - 0s 133us/step - loss: 0.3211 - accuracy: 0.8393 - val_loss: 0.3467 - val_accuracy: 0.8264
Epoch 830/1000
336/336 [==============================] - 0s 116us/step - loss: 0.3133 - accuracy: 0.8423 - val_loss: 0.4156 - val_accuracy: 0.7500
Epoch 831/1000
336/336 [==============================] - 0s 139us/step - loss: 0.3174 - accuracy: 0.8333 - val_loss: 0.3534 - val_accuracy: 0.8264
Epoch 832/1000
336/336 [==============================] - 0s 120us/step - loss: 0.3173 - accuracy: 0.8452 - val_loss: 0.3473 - val_accuracy: 0.8264
Epoch 833/1000
336/336 [==============================] - 0s 249us/step - loss: 0.3164 - accuracy: 0.8274 - val_loss: 0.3851 - 

336/336 [==============================] - 0s 151us/step - loss: 0.3234 - accuracy: 0.8333 - val_loss: 0.3438 - val_accuracy: 0.8333
Epoch 883/1000
336/336 [==============================] - 0s 161us/step - loss: 0.3201 - accuracy: 0.8304 - val_loss: 0.3475 - val_accuracy: 0.8333
Epoch 884/1000
336/336 [==============================] - 0s 128us/step - loss: 0.3118 - accuracy: 0.8423 - val_loss: 0.3466 - val_accuracy: 0.8333
Epoch 885/1000
336/336 [==============================] - 0s 121us/step - loss: 0.3090 - accuracy: 0.8423 - val_loss: 0.3456 - val_accuracy: 0.8333
Epoch 886/1000
336/336 [==============================] - 0s 125us/step - loss: 0.3147 - accuracy: 0.8304 - val_loss: 0.3451 - val_accuracy: 0.8333
Epoch 887/1000
336/336 [==============================] - 0s 150us/step - loss: 0.3139 - accuracy: 0.8423 - val_loss: 0.3446 - val_accuracy: 0.8333
Epoch 888/1000
336/336 [==============================] - 0s 136us/step - loss: 0.3323 - accuracy: 0.8274 - val_loss: 0.3491 - 

336/336 [==============================] - 0s 146us/step - loss: 0.3113 - accuracy: 0.8423 - val_loss: 0.3494 - val_accuracy: 0.8333
Epoch 938/1000
336/336 [==============================] - 0s 161us/step - loss: 0.3124 - accuracy: 0.8423 - val_loss: 0.3479 - val_accuracy: 0.8333
Epoch 939/1000
336/336 [==============================] - 0s 169us/step - loss: 0.3145 - accuracy: 0.8423 - val_loss: 0.3444 - val_accuracy: 0.8333
Epoch 940/1000
336/336 [==============================] - 0s 138us/step - loss: 0.3121 - accuracy: 0.8423 - val_loss: 0.3441 - val_accuracy: 0.8333
Epoch 941/1000
336/336 [==============================] - 0s 145us/step - loss: 0.3088 - accuracy: 0.8423 - val_loss: 0.3478 - val_accuracy: 0.8333
Epoch 942/1000
336/336 [==============================] - 0s 147us/step - loss: 0.3146 - accuracy: 0.8423 - val_loss: 0.3448 - val_accuracy: 0.8333
Epoch 943/1000
336/336 [==============================] - 0s 130us/step - loss: 0.3114 - accuracy: 0.8423 - val_loss: 0.3448 - 

336/336 [==============================] - 0s 422us/step - loss: 0.3137 - accuracy: 0.8423 - val_loss: 0.3445 - val_accuracy: 0.8333
Epoch 993/1000
336/336 [==============================] - 0s 303us/step - loss: 0.3137 - accuracy: 0.8304 - val_loss: 0.3515 - val_accuracy: 0.8333
Epoch 994/1000
336/336 [==============================] - 0s 237us/step - loss: 0.3110 - accuracy: 0.8393 - val_loss: 0.3483 - val_accuracy: 0.8333
Epoch 995/1000
336/336 [==============================] - 0s 138us/step - loss: 0.3116 - accuracy: 0.8423 - val_loss: 0.3437 - val_accuracy: 0.8333
Epoch 996/1000
336/336 [==============================] - 0s 128us/step - loss: 0.3203 - accuracy: 0.8363 - val_loss: 0.3477 - val_accuracy: 0.8333
Epoch 997/1000
336/336 [==============================] - 0s 133us/step - loss: 0.3268 - accuracy: 0.8393 - val_loss: 0.3472 - val_accuracy: 0.8333
Epoch 998/1000
336/336 [==============================] - 0s 119us/step - loss: 0.3127 - accuracy: 0.8423 - val_loss: 0.3430 - 

In [68]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

144/144 [==============================] - 0s 75us/step
over-sampling test accuracy: 83.33%


In [51]:
############ Logistic Regression ############

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [53]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 100.00%


In [55]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 97.92%


In [57]:
############## Random Forest ##############

In [58]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [59]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 99.24%


In [60]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [61]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [62]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [63]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         0.96721311 1.         1.         1.        ]
0.9934426229508198


In [64]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98529412 0.97058824 1.         1.         0.96969697]
0.9851158645276292
